In [ ]:
%config Completer.use_jedi = False

In [ ]:
!nvidia-smi

In [ ]:
"""%%capture

!rm -Rf pixray
!git clone --recursive https://github.com/paulasquin/pixray.git
!pip install pillow basicsr pydiffvg
!pip install --upgrade pillow
!pip install -r pixray/requirements.txt
"""

In [ ]:
!mkdir -p steps
!mkdir -p models

import sys
# sys.path.append("pixray")
# import pixray


In [ ]:
prompts="deep space planet #artstation" #@param {type:"string"}
optimiser="Adam" #@param ["Adam", "DiffGrad"]
quality="normal" #@param ["draft", "normal", "better", "best"]
init_noise="gradient" #@param ["pixels", "gradient" , "snow"]
drawer = "clipdraw" #@param ["vqgan", "clipdraw", "pixel", "line_sketch"]
# clip_models='RN50,ViT-B/16,SLIP_VITB16'
batches=2
num_cuts=2
scale = "1.5" #@param {type:"string"}
display_every=10

scale = float(scale)
width=f"{int(1080//scale)}"
height=f"{int(1920//scale)}"
# height = int(float(width) * 5.0 / 4)
run_name = prompts.replace(" ", "_").replace("#", "").replace("|", "")
outdir=f"aiart/{run_name}"
print(f"{run_name}: {width}x{height}")

In [ ]:
import os
if not os.path.isdir(outdir):
    os.makedirs(outdir)

In [ ]:
# !pip install -r requirements.txt
!pip install -e v-diffusion-pytorch/
!pip install -e .


In [ ]:
import diffusion

In [ ]:
from pixray import pixray

In [ ]:
pixray.reset_settings()
pixray.add_settings(prompts=prompts, init_noise=init_noise, outdir=outdir, save_every=1, make_video=True, vector_prompts="textoff",
                   size=[int(width), int(height)], quality=quality, save_intermediates=True, custom_loss="aesthetic", optimiser=optimiser,
                   display_every=display_every, num_cuts=num_cuts, batches=batches)
settings = pixray.apply_settings()
pixray.do_init(settings)

In [ ]:
pixray.do_run(settings)

In [ ]:
# import pixray
pixray.step_to_video('steps')

In [ ]:
!rm aiart.zip
!cp steps/*.mp4 $outdir
!zip -r aiart.zip aiart

In [ ]:
!git clone https://github.com/iamleot/transferwee.git
!python transferwee/transferwee.py upload airart.zip -m "airart"

In [ ]:
!cp aiart.zip /kaggle/working/aiart.zip
from IPython.display import FileLink
FileLink(r'aiart.zip')